In [19]:
import pickle
import numpy as np
import pandas as pd

In [20]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [21]:
predicted_test = test.copy()

feature_1= 'x'
feature_2='y'

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_1_2/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    angle_2 = test[feature_2].iloc[i]
    X=np.array([angle_1,angle_2]).T
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    
# model_0.score_samples(angles_0)

In [22]:
print(predicted_class)
print(true_label)

print(len(predicted_class))
print(len(true_label))

[4, 1, 9, 3, 0, 8, 0, 4, 5, 0, 8, 4, 1, 2, 6, 4, 4, 4, 1, 2, 8, 0, 5, 0, 5, 2, 0, 0, 5, 2, 7, 5, 2, 0, 0, 4, 2, 2, 2, 9, 3, 2, 9, 5, 0, 7, 3, 5, 8, 6, 9, 4, 6, 3, 3, 1, 0, 1, 6, 8, 7, 4, 6, 9, 9, 4, 1, 0, 0, 4, 5, 3, 6, 1, 6, 7, 4, 2, 9, 0, 7, 9, 8, 7, 2, 8, 7, 6, 3, 8, 4, 1, 0, 4, 1, 9, 5, 2, 8, 7, 3, 0, 6, 8, 5, 5, 1, 1, 1, 7, 8, 1, 5, 8, 6, 6, 4, 3, 6, 4, 5, 5, 8, 5, 0, 9, 6, 3, 0, 6, 0, 2, 9, 9, 7, 8, 5, 1, 6, 9, 8, 3, 5, 4, 1, 0, 7, 5, 8, 7, 4, 6, 5, 1, 8, 8, 5, 0, 2, 7, 8, 6, 3, 4, 4, 4, 8, 8, 0, 4, 4, 9, 0, 8, 6, 4, 0, 9, 8, 0, 6, 1, 4, 1, 8, 2, 3, 7, 2, 0, 5, 5, 2, 5, 8, 1, 5, 6, 0, 7, 6, 6, 0, 1, 1, 3, 7, 8, 5, 3, 6, 4, 0, 4, 3, 7, 4, 8, 0, 3, 0, 0, 3, 2, 5, 3, 8, 8, 6, 0, 1, 4, 1, 2, 6, 8, 7, 1, 0, 8, 0, 5, 7, 2, 1, 2, 5, 6]
[4, 1, 9, 3, 5, 8, 0, 4, 6, 0, 8, 4, 7, 2, 6, 4, 7, 3, 1, 2, 8, 0, 9, 0, 5, 3, 6, 4, 5, 2, 4, 5, 2, 0, 0, 4, 2, 1, 2, 9, 3, 2, 9, 3, 1, 7, 5, 5, 8, 6, 7, 4, 5, 3, 3, 1, 0, 9, 6, 8, 9, 0, 6, 9, 9, 8, 1, 0, 2, 4, 5, 2, 6, 9, 3, 4, 4, 2, 6, 0, 9, 9, 8, 7, 2,

In [23]:
from sklearn.metrics.cluster import contingency_matrix 


In [24]:
matrix=contingency_matrix(true_label, predicted_class)

In [25]:
matrix

array([[24,  0,  0,  0,  5,  1,  1,  1,  2,  0],
       [ 1, 18,  1,  0,  0,  0,  0,  2,  1,  0],
       [ 1,  2, 16,  3,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  2, 12,  1,  2,  2,  1,  2,  0],
       [ 1,  0,  0,  0, 20,  1,  0,  2,  1,  0],
       [ 1,  0,  1,  3,  1, 15,  2,  0,  2,  0],
       [ 2,  0,  1,  0,  0,  2, 19,  0,  0,  1],
       [ 1,  3,  0,  0,  1,  1,  0,  9,  2,  4],
       [ 2,  0,  0,  0,  1,  2,  0,  1, 18,  0],
       [ 2,  2,  0,  1,  0,  4,  1,  3,  3, 10]])

In [26]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.6491935483870968


In [27]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.8467741935483871
